In [19]:
TEST='test'

In [7]:
!python ./exp/nb_00.py first.ipynb

In [26]:
import operator
def test(a,b,cmp,cname=None):
    if cname is None : cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

In [27]:
test_eq(TEST,'test')

In [1]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from torch.nn import init


MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [2]:
def get_data():
    
    path=datasets.download_data(MNIST_URL,ext='.gz');path
    with gzip.open(path,'rb') as f:
        ((x_train,y_train),(x_valid,y_valid),_)=pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train,y_train,x_valid,y_valid))
def normalize(x,m,s): return (x-m)/s
def lin(x,w,b): return x@w +b
def relu(x): return x.clamp_min(0.)-0.7

In [3]:
x_train,y_train,x_valid,y_valid=get_data()

In [4]:
n,c=x_train.shape
x_train,x_train.shape,y_train,y_train.shape,y_train.min(),y_train.max()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([50000, 784]),
 tensor([5, 0, 4,  ..., 8, 4, 8]),
 torch.Size([50000]),
 tensor(0),
 tensor(9))

In [5]:
assert n==y_train.shape[0]==50000
test_eq(c,28*28),
test_eq(y_train.min(),0)
test_eq(y_train.max(),9)

NameError: name 'test_eq' is not defined

In [ ]:
img=x_train[0]
img.view(28,28).type()

In [ ]:
plt.imshow(img.view(28,28),cmap='gray')

In [ ]:
weigths=torch.randn(784,10)

In [ ]:
bias=torch.zeros(10)

In [ ]:
def matmul(a,b): return a.matmul(b)

In [ ]:
m1=x_valid[:5]
m2=weigths

In [ ]:
m1.shape,m2.shape

In [ ]:
 %timeit -n 10 _ = matmul(m1,m2)

In [ ]:
t1.shape

# TESTING BROADCAST

In [ ]:
a=torch.tensor([[1,2,3]])
b=torch.tensor([[1],[2],[3]])

In [ ]:
a.shape,b.shape

In [ ]:
a,b

In [ ]:
a[:,None]*b

# Continuew

In [32]:
train_mean,train_std=x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [33]:
x_train_normalized=normalize(x_train,train_mean,train_std)
x_valid_normalized=normalize(x_valid,train_mean,train_std)

In [34]:
n,m=x_train.shape
c=y_train.max()+1
n,m,c

(50000, 784, tensor(10.))

# Basic architecture

In [35]:
num_hidden=50

In [36]:
w1=torch.randn(m,num_hidden)*math.sqrt(2./m)
# w1=torch.randn(m,num_hidden)
# init.kaiming_normal_(w1,mode='fan_out')
b1=torch.zeros(num_hidden)
w2=torch.randn(num_hidden,1)*math.sqrt(2./num_hidden)
# w2=torch.randn(num_hidden,1)
# init.kaiming_normal_(w2,mode='fan_out')
b2=torch.zeros(1)

In [37]:
x_valid@w1

tensor([[ 0.2510, -1.1889, -0.4073,  ..., -0.0498, -0.1602,  0.3075],
        [ 0.2429, -0.9035, -0.7607,  ...,  0.0306, -0.7968, -0.1197],
        [ 0.0198, -0.2687, -0.6766,  ..., -0.1851, -0.4481,  0.2492],
        ...,
        [ 0.1700, -0.7712, -0.1871,  ...,  0.1296, -1.1240, -0.0064],
        [ 0.4680, -0.1063, -0.4111,  ...,  0.4098, -0.0735, -0.2785],
        [ 0.2890, -0.4414,  0.0943,  ...,  0.8416,  0.2050,  0.7784]])

In [38]:
t=relu(lin(x_valid,w1,b1))

In [39]:
t.shape

torch.Size([10000, 50])

In [40]:
t.mean(),t.std()

(tensor(-0.4894), tensor(0.3062))

In [41]:
def model(x):
    l1=lin(x,w1,b1)
#     print(l1.shape)
    l2=relu(l1)
#     print(l2.shape)
    l3=lin(l2,w2,b2)
    return l3

In [42]:
%timeit -n 10 _=model(x_valid)

8.22 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
model(x_valid).shape

torch.Size([10000, 1])

In [44]:
def mse(output,target): return (output.squeeze(1)-target).pow(2).mean()

In [45]:
y_train,y_valid=y_train.float(),y_valid.float()

In [46]:
preds=model(x_train)

In [47]:
mse(preds,y_train)

tensor(37.7787)

In [48]:
def mse_grad(inp,tar):
    inp.g=2*(inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0]

In [49]:
def relu_grad(inp,out):
    inp.g=(inp>0).float()*out.g

In [50]:
def lin_grad(inp,out,w,b):
    inp.g=out.g@w.t()
    w.g=(inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g=out.g.sum(0)

In [51]:
def forward_backward(inp,targ):
    l1=inp@w1+b1
    l2=relu(l1)
    out=l2@w2+b2
    loss=mse(out,tar)
    
    mse_grad(out,targ)
    lin_grad(l2,out,w2,b2)
    relu_grad(l1,l2)
    

# Refactor

In [52]:
class ReLU():
    def __call__(self,inp):
        self.inp=inp
        self.out=inp.clamp_min(0.)-0.5
        return self.out
    def backward(self): self.inp.g= (self.inp>0).float()*self.out.g

In [53]:
class Lin():
    def __init__(self,w,b):
        self.w=w
        self.b=b
    
    def __call__(self,inp):
        self.inp=inp
        self.out=inp@self.w+self.b
        return self.out
    def backward(self):
        self.inp.g=selg.out.g@self.w.t()
        self.w.g=(self.inp.unsqueeze(-1)*self.out.g.unsqueeze(1)).sum(0)
        self.b.g=self.out.g.sum(0)
        
        

In [54]:
def lin_grad(inp,out,w,b):
    inp.g=out.g@w+b
    w.g=(inpunqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g=out.g.sum(0)

In [55]:
class MSE():
    def __call__(self,pred,targ):
        self.pred=pred
        self.targ=targ
        self.out=(pred.squeeze()-targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.pred.g=2.*(self.pred.squeeze()-self.targ).unsqueeze(-1)/self.targ.shape[0]

In [56]:
class Model():
    def __init__(self,w1,b1,w2,b2):
        self.layers=[Lin(w1,b1),ReLU(),Lin(w2,b2)]
        self.loss=MSE()
        
    def __call__(self,x,targ):
        for l in self.layers: x=l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
        

In [29]:
w1.g,b1.g,w2.g,b2.g=[None]*4
model=Model(w1,b1,w2,b2)

In [30]:
%time loss=model(x_train_normalized,y_train)

NameError: name 'x_train_normalized' is not defined